In [2]:
# 2.2 数据预处理
import os
import pandas as pd

data_file = os.path.join("..","data","house_tiny.csv")
data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [4]:
inputs, outputs = data.iloc[:,0:2], data.iloc[:,2]
# inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,Alley
0,NaN,Pave
1,2.0,NaN
2,4.0,NaN
3,NaN,NaN


In [11]:
inputs = pd.get_dummies(inputs,dtype=float, dummy_na=True)
inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,True,False
1,2.0,False,True
2,4.0,False,True
3,3.0,False,True


In [14]:
# 转换为张量
import torch
x=torch.tensor(inputs.to_numpy(dtype=float))
print(x)
y=torch.tensor(outputs.to_numpy(dtype=float))
print(y)

tensor([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]], dtype=torch.float64)
tensor([127500., 106000., 178100., 140000.], dtype=torch.float64)


In [15]:
# 2.3 线性
import torch
x=torch.tensor(3.0)
y=torch.tensor(2.0)
x+y,x*y,x/y,x**y

(tensor(5.), tensor(6.), tensor(1.5000), tensor(9.))

In [18]:
A=torch.arange(20).reshape((4,5))
A

tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])

In [19]:
A.T

tensor([[ 0,  5, 10, 15],
        [ 1,  6, 11, 16],
        [ 2,  7, 12, 17],
        [ 3,  8, 13, 18],
        [ 4,  9, 14, 19]])

In [21]:
X=torch.arange(24).reshape(2,3,4)
Y=X.clone()

In [1]:
# # FTP 测试
# from ftplib import FTP
# import os

# def download_ftp_file(host, username, password, remote_file_path, local_file_path):
#     try:
#         # 连接FTP服务器并登录
#         with FTP(host) as ftp:
#             ftp.login(username, password)
#             # 启用被动模式（适用于防火墙环境）
#             ftp.set_pasv(True)
            
#             # 下载文件（二进制模式）
#             with open(local_file_path, 'wb') as local_file:
#                 ftp.retrbinary(f'RETR {remote_file_path}', local_file.write)
            
#             print(f"文件下载成功: {local_file_path}")
#     except Exception as e:
#         print(f"下载失败: {str(e)}")

# # 使用示例
# if __name__ == "__main__":
#     # 配置FTP服务器信息
#     ftp_host = "192.168.1.100"  # 替换为实际FTP服务器IP
#     ftp_username = "your_username"  # 替换为实际用户名
#     ftp_password = "your_password"  # 替换为实际密码
#     remote_file = "/remote/path/to/file.txt"  # 替换为远程文件路径
#     local_file = "C:\local\save\path\file.txt"  # 替换为本地保存路径
    
#     download_ftp_file(ftp_host, ftp_username, ftp_password, remote_file, local_file)


from ftplib import FTP
import os

def traverse_ftp_files(host, username, password, start_dir='/'):
    file_list = []
    
    def recursive_list(ftp, current_dir):
        try:
            # 切换到当前目录
            ftp.cwd(current_dir)
            # 获取目录下所有条目
            entries = ftp.nlst()
            
            for entry in entries:
                # 排除默认的.和..目录
                if entry in ['.', '..']:
                    continue
                
                entry_path = f'{current_dir}/{entry}' if current_dir != '/' else f'/{entry}'
                try:
                    # 尝试切换到子目录 - 如果成功则是目录
                    ftp.cwd(entry)
                    # 递归遍历子目录
                    recursive_list(ftp, entry_path)
                    # 切换回父目录
                    ftp.cwd('..')
                except Exception:
                    # 如果切换失败则是文件
                    file_list.append(entry_path)
        except Exception as e:
            print(f"无法访问目录 {current_dir}: {str(e)}")
    
    try:
        with FTP(host) as ftp:
            ftp.login(username, password)
            ftp.set_pasv(True)
            recursive_list(ftp, start_dir)
            
        return file_list
    except Exception as e:
        print(f"FTP连接失败: {str(e)}")
        return []

# 使用示例
if __name__ == "__main__":
    ftp_host = "10.64.10.222"
    ftp_username = "51952"
    ftp_password = "atxks@0730"
    
    all_files = traverse_ftp_files(ftp_host, ftp_username, ftp_password,start_dir="/BCF专用保存路径")
    
    print("FTP服务器上的所有文件:")
    for file_path in all_files:
        print(file_path)

KeyboardInterrupt: 